In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
# init parameters
training_rate   = 0.01
training_epochs = 25
batch_size      = 100

n_input   = mnist.train.images.shape[1] 
n_classes = mnist.train.labels.shape[1]

n_L1 = 256
n_L2 = 256

In [ ]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# first hidden layer, L1 neurons
W1 = tf.Variable(tf.random_normal([n_input, n_L1]))
b1 = tf.Variable(tf.random_normal([n_L1]))

# second hidden layer, L2 neurons
W2 = tf.Variable(tf.random_normal([n_L1, n_L2]))
b2 = tf.Variable(tf.random_normal([n_L2]))

# output layer, K neurons
W3 = tf.Variable(tf.random_normal([n_L2, n_classes]))
b3 = tf.Variable(tf.random_normal([n_classes]))

# construct nn
hidden_layer1 = tf.nn.relu(tf.add(tf.matmul(x, W1), b1))
hidden_layer2 = tf.nn.relu(tf.add(tf.matmul(hidden_layer1, W2), b2))
output_layer  = tf.matmul(hidden_layer2, W3) + b3

In [ ]:
# setup loss and optimizer
softmax_result = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_layer)
cross_entropy = tf.reduce_mean(softmax_result)
optimizer = tf.train.GradientDescentOptimizer(training_rate)
train_step = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_step, cross_entropy],
                            feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy: %4f" % accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))